# 🐐 Knowledge Graph-Enhanced LLM for Goat Disease QA System

## 📑 論文實驗流程實作
**論文標題:** A Knowledge Graph-Enhanced Large Language Model Framework for Goat Disease Question Answering System

---

## 🎯 實驗設計概覽

本筆記本實作論文中的 **4 階段實驗設計 (4-Phase Experimental Design)**:

### **Phase 1: 索引優化 (Indexing Optimization)**
- 消融實驗測試不同的 Chunk Size 與 Overlap
- 輸出: `ablation_chunking.csv`

### **Phase 2: 基準線與診斷 (Baseline & Diagnosis)**
- 計算「優化前」的圖譜品質指標
- 比較 Vector-Only RAG vs. Initial Graph RAG
- 輸出: `baseline_comparison.csv`

### **Phase 3: 結構優化 (Structural Optimization)**
- 執行 Algorithm 1 (星型拓樸偵測)
- 執行 Algorithm 2 (圖譜增強)
- 目標: 關係密度 (Relation Density) > 2.0

### **Phase 4: 檢索消融 (Retrieval Ablation)**
- 測試不同的 Hop Counts 與 Top-K 值
- 輸出: `retrieval_ablation.csv` + `final_experiment.jsonl`

---

## 📦 輸出格式

- **JSONL**: 詳細日誌（每個問題的完整軌跡）
- **CSV**: 指標統計表（用於製作消融實驗表格）

---

## 🚀 快速開始

1. 確保 Neo4j 與 Ollama 服務已啟動
2. 依序執行下方所有 Cell
3. 預估總執行時間：2-4 小時（取決於問題數量）

In [18]:
# ============================================================
# 📋 第 1 步：配置設定 (Configuration Setup)
# ============================================================

import os
from pathlib import Path
from neo4j import GraphDatabase

# 基礎路徑
BASE_DIR = Path.cwd()
KNOWLEDGE_BASE_PATH = BASE_DIR / "goat_data_text collection-1.2-eng.txt"
QUESTION_DATASET_PATH = BASE_DIR / "topic.csv"

# 🔥 自定義三元組抽取 Prompt（高密度知識圖譜）
TRIPLE_PROMPT_TEMPLATE = """
You are an expert knowledge graph engineer. Your task is to extract **explicit and implicit semantic triples** from the text to build a high-density knowledge graph in {language}.

🎯 **Core Objectives (Target Density > 1.8)**:
1. **Zero Isolated Nodes**: Ensure every entity has 2+ connections. Transform weak entities into connected hubs.
2. **Deep Implicit Mining**: Extract causal, functional, and attribute relationships hidden within and across sentences.
3. **Strict Relation Types**: Use specific predicates (e.g., 'causes', 'contains') instead of vague ones (e.g., 'related').
4. **Attribute as Relations**: Treat numbers, states, time, and types as relation tails (e.g., (goat, weight_is, 45kg)).

═══════════════════════════════════════════════════════════════════

## 🛠️ Extraction Strategy Checklist (Must Execute)

### 1. 🔍 Explicit & Implicit Relationship Mining
* **Layer 1 (Explicit)**: Extract directly stated relations (A causes B).
* **Layer 2 (Intra-sentence Implicit)**: Infer hidden links (Subject → Action → Outcome).
    * *Example*: "Vitamin A deficiency causes night blindness." → Extract (Vitamin_A_deficiency, causes, night_blindness) AND (night_blindness, symptom_of, Vitamin_A_deficiency).
* **Layer 3 (Cross-sentence Implicit)**: Connect entities across sentences via shared context.
    * *Example*: "Goats lack Vitamin A. It causes blindness." → Connect (goat, deficient_in, Vitamin_A) AND (Vitamin_A, prevents, blindness).

### 2. 🔢 Attribute & Data Extraction (Crucial for Density)
* **Numerical**: (feed, protein_content_is, 18%), (goat, weight_is, 45kg)
* **State/Characteristic**: (sick_goat, state_is, lethargic), (lesion, color_is, red)
* **Time/Frequency**: (treatment, duration_is, 7_days), (medication, frequency_is, twice_daily)
* **Classification**: (goat, breed_is, Boer), (pneumonia, type_is, respiratory_disease)

### 3. 🔗 Coreference Resolution (Mandatory)
* **Resolve Pronouns**: Replace 'it', 'this', 'that', 'the animal' with the specific entity name.
    * *Bad*: (it, causes, death)
    * *Good*: (viral_infection, causes, death)
* **Restore Omitted Subjects**: If a sentence starts with a verb, link it to the subject from the previous sentence.

### 4. 📝 Standardized Relation Types (Use These Verbs)
* **Causality**: causes, leads_to, triggers, induces, results_in, prevents, inhibits
* **Composition**: contains, comprised_of, part_of, ingredient_is
* **Attribute**: weight_is, length_is, color_is, state_is, located_at, occurs_at
* **Hierarchy**: is_a, belongs_to, type_of, classified_as
* **Function**: used_for, treats, improves, requires, depends_on
* **🚫 BANNED**: related_to, associated_with, has, is (unless 'is_a'), involving.

═══════════════════════════════════════════════════════════════════

## ✅ Output Format
Output **ONLY** a JSON array of triples. No markdown, no explanations.

**Example**:
[
  {{"head": "goat", "relation": "deficient_in", "tail": "vitamin_A"}},
  {{"head": "vitamin_A_deficiency", "relation": "causes", "tail": "night_blindness"}},
  {{"head": "night_blindness", "relation": "symptom_of", "tail": "nutritional_deficiency"}},
  {{"head": "goat", "relation": "weight_is", "tail": "45kg"}}
]

**Text to Extract**:
{chunk}
"""

CONFIG = {
    # ==========================================
    # A. 環境與基礎設施 (使用者提供)
    # ==========================================
    "infrastructure": {
        "neo4j_uri": os.environ.get("NEO4J_URI", "bolt://localhost:7687"),
        "neo4j_auth": (
            os.environ.get("NEO4J_USER", "neo4j"),
            os.environ.get("NEO4J_PASSWORD", "neo4jgoat")
        ),
        "ollama_host": os.environ.get("OLLAMA_HOST", "http://localhost:11434"),
        "dataset_id": KNOWLEDGE_BASE_PATH.stem.replace(" ", "_"),
        "vector_index_name": "chunk_embeddings",
        "fulltext_index_name": "chunk_text_fts",
    },

    # ==========================================
    # B. 模型設定
    # ==========================================
    "models": {
        "llm_model": "deepseek-r1:8b-llama-distill-q4_K_M",
        "graph_create_model": "deepseek-r1:8b-llama-distill-q4_K_M",
        "embed_model": "nomic-embed-text",
        "answer_language": "english"
    },

    # ==========================================
    # C. 生成參數（優化以避免 CUDA OOM）
    # ==========================================
    "generation": {
        "temperature": 0.0,
        "max_questions": 20,      # 🔥 減少測試數量
        "context_window": 4096,
        "batch_size": 1,          # 🔥 單次處理一個 chunk
        "max_workers": 1          # 🔥 避免並行造成記憶體溢出
    },

    # ==========================================
    # D. 第一階段：索引消融網格（簡化測試）
    # ==========================================
    "indexing_grid": [
        {"chunk_size": 2048, "overlap": 512},  # 僅測試最佳配置
    ],
    "optimal_indexing": {"chunk_size": 2048, "overlap": 512},

    # ==========================================
    # E. 第三階段：圖譜優化參數
    # ==========================================
    "optimization": {
        "hub_threshold_percentile": 95,
        "max_iterations": 2,      # 🔥 減少迭代次數
        "quality_threshold": 2.0
    },

    # ==========================================
    # F. 第四階段：檢索消融網格（簡化測試）
    # ==========================================
    "retrieval_grid": {
        "hop_counts": [1, 2],     # 🔥 減少測試組合
        "top_k_values": [10, 15],
        "rerank_thresholds": [0.6, 0.75]
    }
}

print("✅ 配置載入完成")
print(f"📁 知識庫路徑: {KNOWLEDGE_BASE_PATH}")
print(f"📁 問題集路徑: {QUESTION_DATASET_PATH}")
print(f"🔧 Neo4j URI: {CONFIG['infrastructure']['neo4j_uri']}")
print(f"🤖 推論模型: {CONFIG['models']['llm_model']}")
print(f"📊 索引消融實驗組數: {len(CONFIG['indexing_grid'])}")
print(f"📊 檢索消融實驗組數: {len(CONFIG['retrieval_grid']['hop_counts']) * len(CONFIG['retrieval_grid']['top_k_values'])}")
print(f"🔥 自定義三元組抽取 Prompt 已載入（{len(TRIPLE_PROMPT_TEMPLATE)} 字元）")

✅ 配置載入完成
📁 知識庫路徑: c:\Users\kbllm\Desktop\llm-KB\new\goat_data_text collection-1.2-eng.txt
📁 問題集路徑: c:\Users\kbllm\Desktop\llm-KB\new\topic.csv
🔧 Neo4j URI: bolt://localhost:7687
🤖 推論模型: deepseek-r1:8b-llama-distill-q4_K_M
📊 索引消融實驗組數: 1
📊 檢索消融實驗組數: 4
🔥 自定義三元組抽取 Prompt 已載入（3215 字元）


In [19]:
# ============================================================
# 📦 第 2 步：依賴套件導入與記憶體管理
# ============================================================

import json
import csv
import time
import gc
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List, Dict, Any, Tuple
from collections import defaultdict

# Neo4j & GraphRAG
from neo4j import GraphDatabase

# 🔥 直接使用 ollama 套件（避免 neo4j_graphrag 的整合問題）
import ollama

# 評估指標
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# 🔥 GPU 與記憶體管理
import torch
if torch.cuda.is_available():
    print(f"✅ GPU 可用: {torch.cuda.get_device_name(0)}")
    print(f"   記憶體: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    # 設置記憶體優化
    torch.cuda.empty_cache()
    gc.collect()
else:
    print("⚠️ GPU 不可用，將使用 CPU")

def clear_memory():
    """清理 GPU 和系統記憶體"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# 🔥 封裝 Ollama 客戶端
class OllamaEmbedder:
    def __init__(self, model: str):
        self.model = model
        self.client = ollama.Client()
    
    def embed_query(self, text: str) -> List[float]:
        """生成文字嵌入"""
        response = self.client.embeddings(model=self.model, prompt=text)
        return response['embedding']
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """批次生成嵌入"""
        return [self.embed_query(text) for text in texts]

class OllamaLLMWrapper:
    def __init__(self, model_name: str, temperature: float = 0.0):
        self.model_name = model_name
        self.temperature = temperature
        self.client = ollama.Client()
    
    def invoke(self, prompt: str) -> Any:
        """呼叫 LLM 生成回應"""
        response = self.client.generate(
            model=self.model_name,
            prompt=prompt,
            options={"temperature": self.temperature}
        )
        # 包裝成類似 langchain 的格式
        class Response:
            def __init__(self, content):
                self.content = content
        return Response(response['response'])

print("✅ 所有依賴套件導入完成")

✅ GPU 可用: NVIDIA GeForce RTX 4090
   記憶體: 23.99 GB
✅ 所有依賴套件導入完成


In [20]:
# ============================================================
# 🧪 環境驗證測試 (Environment Verification)
# ============================================================

print("=" * 60)
print("環境驗證測試")
print("=" * 60)

# 測試 1: Neo4j 連接
print("\n[1/4] 測試 Neo4j 連接...")
try:
    test_driver = GraphDatabase.driver(
        CONFIG['infrastructure']['neo4j_uri'],
        auth=CONFIG['infrastructure']['neo4j_auth']
    )
    with test_driver.session() as session:
        result = session.run("RETURN 1 as test")
        assert result.single()["test"] == 1
    test_driver.close()
    print("✅ Neo4j 連接成功")
except Exception as e:
    print(f"❌ Neo4j 連接失敗: {e}")

# 測試 2: Ollama 服務
print("\n[2/4] 測試 Ollama 服務...")
try:
    test_embedder = OllamaEmbedder(model=CONFIG['models']['embed_model'])
    test_vector = test_embedder.embed_query("test")
    assert len(test_vector) > 0
    print(f"✅ Ollama Embeddings 正常 (維度: {len(test_vector)})")
except Exception as e:
    print(f"❌ Ollama Embeddings 失敗: {e}")

try:
    test_llm = OllamaLLMWrapper(
        model_name=CONFIG['models']['llm_model'],
        temperature=CONFIG['generation']['temperature']
    )
    response = test_llm.invoke("Say 'Hello'")
    print(f"✅ Ollama LLM 正常 (回應: {response.content[:50]}...)")
except Exception as e:
    print(f"❌ Ollama LLM 失敗: {e}")

# 測試 3: 檔案存在性
print("\n[3/4] 測試資料檔案...")
if KNOWLEDGE_BASE_PATH.exists():
    print(f"✅ 知識庫檔案存在 ({KNOWLEDGE_BASE_PATH.stat().st_size / 1024:.2f} KB)")
else:
    print(f"❌ 知識庫檔案不存在: {KNOWLEDGE_BASE_PATH}")

if QUESTION_DATASET_PATH.exists():
    df = pd.read_csv(QUESTION_DATASET_PATH)
    print(f"✅ 問題集檔案存在 ({len(df)} 個問題)")
else:
    print(f"❌ 問題集檔案不存在: {QUESTION_DATASET_PATH}")

# 測試 4: 計算資源
print("\n[4/4] 測試計算資源...")
import psutil
print(f"✅ CPU 核心數: {psutil.cpu_count()}")
print(f"✅ 可用記憶體: {psutil.virtual_memory().available / (1024**3):.2f} GB")

print("\n" + "=" * 60)
print("✅ 環境驗證完成！可以開始執行實驗")
print("=" * 60)

環境驗證測試

[1/4] 測試 Neo4j 連接...
✅ Neo4j 連接成功

[2/4] 測試 Ollama 服務...
✅ Ollama Embeddings 正常 (維度: 768)
✅ Ollama LLM 正常 (回應: Hello! How can I assist you today? 😊...)

[3/4] 測試資料檔案...
✅ 知識庫檔案存在 (679.61 KB)
✅ 問題集檔案存在 (15 個問題)

[4/4] 測試計算資源...
✅ CPU 核心數: 28
✅ 可用記憶體: 37.26 GB

✅ 環境驗證完成！可以開始執行實驗


In [21]:
# ============================================================
# 🔧 第 3 步：核心工具函數
# ============================================================

# ------------------------------------------------------------
# 3.1 系統提示詞 (Section 3.4 Knowledge Grounding)
# ------------------------------------------------------------

GRAPH_RAG_SYSTEM_PROMPT = """
You are a domain expert in Goat Disease Diagnosis.
Answer the question based STRICTLY on the provided Knowledge Graph Context.

### ⛔ Constraints:
1. **No Hallucination**: Do NOT use internal knowledge. If the answer is not in the context, say "Insufficient information."
2. **Traceability**: Explicitly cite the relations used (e.g., "According to [Disease -> has_symptom -> Symptom]...").
3. **Format**: Use concise SVO (Subject-Verb-Object) sentences.

### 🔍 Context:
{context_str}

### 👤 Question:
{query_str}
"""

# ------------------------------------------------------------
# 3.2 數據記錄函數
# ------------------------------------------------------------

def save_jsonl_log(record: Dict[str, Any], filename: str):
    """
    儲存單一問題的詳細日誌至 JSONL 檔案
    
    Args:
        record: 包含問題、檢索結果、答案、指標等的字典
        filename: 輸出檔案名稱
    """
    with open(filename, 'a', encoding='utf-8') as f:
        f.write(json.dumps(record, ensure_ascii=False) + '\n')

def save_csv_metric(row_dict: Dict[str, Any], filename: str):
    """
    儲存實驗批次的統計指標至 CSV 檔案
    
    Args:
        row_dict: 包含階段、參數、平均指標等的字典
        filename: 輸出檔案名稱
    """
    file_exists = Path(filename).exists()
    
    with open(filename, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=row_dict.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(row_dict)

# ------------------------------------------------------------
# 3.3 評估指標計算
# ------------------------------------------------------------

def calculate_f1_score(predicted: str, reference: str) -> float:
    """計算基於詞彙的 F1 分數"""
    pred_tokens = set(predicted.lower().split())
    ref_tokens = set(reference.lower().split())
    
    if len(pred_tokens) == 0 or len(ref_tokens) == 0:
        return 0.0
    
    common = pred_tokens & ref_tokens
    precision = len(common) / len(pred_tokens) if len(pred_tokens) > 0 else 0
    recall = len(common) / len(ref_tokens) if len(ref_tokens) > 0 else 0
    
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def calculate_cosine_similarity(text1: str, text2: str) -> float:
    """計算兩段文字的餘弦相似度"""
    vectorizer = TfidfVectorizer()
    try:
        tfidf = vectorizer.fit_transform([text1, text2])
        return cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]
    except:
        return 0.0

# ------------------------------------------------------------
# 3.4 圖譜品質指標
# ------------------------------------------------------------

def calculate_graph_metrics(driver) -> Dict[str, float]:
    """
    計算圖譜品質指標
    
    Returns:
        - relation_density: 平均每個實體的關係數量
        - weak_entity_ratio: 孤立或低連接實體的比例
    """
    with driver.session() as session:
        # 計算關係密度
        result = session.run("""
            MATCH (e:__Entity__)
            OPTIONAL MATCH (e)-[r]->()
            WITH e, count(r) as degree
            RETURN avg(degree) as avg_degree, 
                   sum(CASE WHEN degree <= 1 THEN 1 ELSE 0 END) * 1.0 / count(e) as weak_ratio
        """)
        record = result.single()
        
        return {
            "relation_density": record["avg_degree"] if record else 0.0,
            "weak_entity_ratio": record["weak_ratio"] if record else 0.0
        }

print("✅ 核心工具函數定義完成")

✅ 核心工具函數定義完成


In [22]:
# ============================================================
# 🏗️ 第 4 步：知識圖譜建構函數（使用自定義 Prompt）
# ============================================================

def simple_text_splitter(text: str, chunk_size: int, chunk_overlap: int) -> List[str]:
    """簡單的文字切塊函數"""
    chunks = []
    start = 0
    text_length = len(text)
    
    while start < text_length:
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - chunk_overlap
    
    return chunks

def extract_json_from_response(content: str) -> List[Dict]:
    """從 LLM 回應中提取 JSON 陣列"""
    import re
    
    # 確保 content 是字串
    if not isinstance(content, str):
        content = str(content)
    
    # 方法 1: 提取 markdown 包裹的 JSON
    if "```json" in content:
        content = content.split("```json")[1].split("```")[0].strip()
    elif "```" in content:
        # 找所有 ``` 區塊
        matches = re.findall(r'```(?:json)?\n?(.*?)\n?```', content, re.DOTALL)
        if matches:
            content = matches[0].strip()
    
    # 方法 2: 尋找第一個 JSON 陣列
    json_match = re.search(r'\[\s*\{.*?\}\s*\]', content, re.DOTALL)
    if json_match:
        content = json_match.group(0)
    
    # 清理註解
    content = '\n'.join([line for line in content.split('\n') 
                         if not line.strip().startswith('//')])
    
    # 嘗試解析
    try:
        data = json.loads(content)
        if isinstance(data, list):
            return data
        return []
    except:
        return []

def build_knowledge_graph_with_custom_prompt(
    text_content: str,
    chunk_size: int,
    chunk_overlap: int,
    driver,
    embedder,
    llm
) -> Dict[str, Any]:
    """
    建構知識圖譜（使用自定義高密度 Prompt）
    
    Args:
        text_content: 知識庫文字內容
        chunk_size: 文字切塊大小
        chunk_overlap: 切塊重疊大小
        driver: Neo4j 驅動器
        embedder: 嵌入模型
        llm: 大型語言模型
        
    Returns:
        包含建構統計資訊的字典
    """
    start_time = time.time()
    
    # 清空現有圖譜
    print("🗑️ 清空現有圖譜...")
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
        session.run("DROP INDEX chunk_embeddings IF EXISTS")
        session.run("DROP INDEX chunk_text_fts IF EXISTS")
    
    # 清理記憶體
    clear_memory()
    
    # 1. 文字切塊
    print(f"✂️ 切塊中 (size={chunk_size}, overlap={chunk_overlap})...")
    chunks = simple_text_splitter(text_content, chunk_size, chunk_overlap)
    print(f"   生成 {len(chunks)} 個 chunks")
    
    # 2. 使用自定義 Prompt 抽取三元組
    print(f"🔍 使用高密度 Prompt 抽取三元組...")
    
    all_entities = set()
    all_relations = []
    successful_chunks = 0
    failed_chunks = 0
    
    for idx, chunk_text in enumerate(chunks):
        if (idx + 1) % 10 == 0:
            print(f"   處理進度: {idx+1}/{len(chunks)} (成功: {successful_chunks}, 失敗: {failed_chunks})")
            clear_memory()
        
        try:
            # 準備 Prompt
            prompt = TRIPLE_PROMPT_TEMPLATE.format(
                language=CONFIG['models']['answer_language'],
                chunk=chunk_text
            )
            
            # 呼叫 LLM 抽取
            response = llm.invoke(prompt)
            content = response.content
            
            # 提取 JSON
            triples = extract_json_from_response(content)
            
            if not triples:
                failed_chunks += 1
                continue
            
            # 3. 儲存到 Neo4j
            with driver.session() as session:
                # 建立 Chunk 節點
                chunk_id = f"chunk_{idx}"
                session.run("""
                    CREATE (c:Chunk {
                        id: $chunk_id,
                        text: $text,
                        index: $index
                    })
                """, {
                    "chunk_id": chunk_id,
                    "text": chunk_text,
                    "index": idx
                })
                
                # 建立實體和關係
                for triple in triples:
                    head = triple.get('head', '').strip()
                    relation = triple.get('relation', '').strip()
                    tail = triple.get('tail', '').strip()
                    
                    if not (head and relation and tail):
                        continue
                    
                    # 建立實體節點
                    session.run("""
                        MERGE (e1:__Entity__ {id: $head})
                        MERGE (e2:__Entity__ {id: $tail})
                    """, {"head": head, "tail": tail})
                    
                    # 建立關係
                    session.run(f"""
                        MATCH (e1:__Entity__ {{id: $head}})
                        MATCH (e2:__Entity__ {{id: $tail}})
                        MERGE (e1)-[r:RELATION {{type: $relation}}]->(e2)
                    """, {"head": head, "tail": tail, "relation": relation})
                    
                    # 連接 Chunk 到實體
                    session.run("""
                        MATCH (c:Chunk {id: $chunk_id})
                        MATCH (e:__Entity__ {id: $entity})
                        MERGE (c)-[:MENTIONS]->(e)
                    """, {"chunk_id": chunk_id, "entity": head})
                    
                    session.run("""
                        MATCH (c:Chunk {id: $chunk_id})
                        MATCH (e:__Entity__ {id: $entity})
                        MERGE (c)-[:MENTIONS]->(e)
                    """, {"chunk_id": chunk_id, "entity": tail})
                    
                    all_relations.append((head, relation, tail))
                    all_entities.add(head)
                    all_entities.add(tail)
            
            successful_chunks += 1
        
        except Exception as e:
            failed_chunks += 1
            if failed_chunks <= 5:  # 只顯示前 5 個錯誤
                print(f"   ⚠️ Chunk {idx} 處理失敗: {str(e)[:100]}")
            continue
    
    print(f"   最終統計: 成功 {successful_chunks}/{len(chunks)}, 失敗 {failed_chunks}")
    
    # 4. 生成嵌入向量
    print(f"🧮 生成嵌入向量...")
    with driver.session() as session:
        chunks_to_embed = session.run("MATCH (c:Chunk) RETURN c.id as id, c.text as text").data()
        
        for idx, chunk in enumerate(chunks_to_embed):
            if (idx + 1) % 50 == 0:
                print(f"   嵌入進度: {idx+1}/{len(chunks_to_embed)}")
                clear_memory()
            
            try:
                embedding = embedder.embed_query(chunk['text'])
                session.run("""
                    MATCH (c:Chunk {id: $id})
                    SET c.embedding = $embedding
                """, {"id": chunk['id'], "embedding": embedding})
            except Exception as e:
                print(f"   ⚠️ 嵌入失敗: {e}")
    
    # 5. 建立索引
    print(f"📇 建立索引...")
    with driver.session() as session:
        session.run(f"""
            CREATE VECTOR INDEX {CONFIG['infrastructure']['vector_index_name']} IF NOT EXISTS
            FOR (c:Chunk) ON c.embedding
            OPTIONS {{indexConfig: {{
                `vector.dimensions`: 768,
                `vector.similarity_function`: 'cosine'
            }}}}
        """)
        
        session.run(f"""
            CREATE FULLTEXT INDEX {CONFIG['infrastructure']['fulltext_index_name']} IF NOT EXISTS
            FOR (c:Chunk) ON EACH [c.text]
        """)
    
    build_time = time.time() - start_time
    
    # 統計資訊
    with driver.session() as session:
        stats = session.run("""
            MATCH (c:Chunk) 
            WITH count(c) as chunk_count
            MATCH (e:__Entity__)
            WITH chunk_count, count(e) as entity_count
            MATCH ()-[r]->()
            RETURN chunk_count, entity_count, count(r) as relation_count
        """).single()
    
    clear_memory()
    
    print(f"✅ 建構完成：{stats['chunk_count']} chunks, {stats['entity_count']} entities, {stats['relation_count']} relations")
    
    return {
        "chunk_size": chunk_size,
        "chunk_overlap": chunk_overlap,
        "build_time": build_time,
        "chunk_count": stats["chunk_count"] if stats else 0,
        "entity_count": stats["entity_count"] if stats else 0,
        "relation_count": stats["relation_count"] if stats else 0
    }

print("✅ 知識圖譜建構函數定義完成（使用自定義高密度 Prompt）")

✅ 知識圖譜建構函數定義完成（使用自定義高密度 Prompt）


In [23]:
# ============================================================
# 🔍 第 5 步：檢索器實作 (Vector & Multi-Hop Retriever)
# ============================================================

from neo4j_graphrag.retrievers.base import Retriever
from neo4j_graphrag.types import RetrieverResultItem, RawSearchResult

class VectorRetriever(Retriever):
    """
    純向量檢索器（不使用圖結構）
    """
    
    def __init__(
        self,
        driver,
        index_name: str,
        embedder,
        neo4j_database: str = None,
    ):
        self.driver = driver
        self.index_name = index_name
        self.embedder = embedder
        self.neo4j_database = neo4j_database
        
    def search(
        self,
        query_text: str = None,
        query_vector: List[float] = None,
        top_k: int = 5,
    ) -> RawSearchResult:
        """執行純向量檢索"""
        
        # 獲取查詢向量
        if query_vector is None and query_text:
            query_vector = self.embedder.embed_query(query_text)
        
        # 向量檢索
        with self.driver.session(database=self.neo4j_database) as session:
            result = session.run(f"""
                CALL db.index.vector.queryNodes(
                    $index_name, 
                    $top_k, 
                    $query_vector
                )
                YIELD node, score
                RETURN node.text as text, 
                       elementId(node) as node_id,
                       score
                ORDER BY score DESC
            """, {
                "index_name": self.index_name,
                "top_k": top_k,
                "query_vector": query_vector
            })
            
            records_list = list(result)  # Neo4j Record objects
        
        # 構建返回結果
        items = [
            RetrieverResultItem(
                content=record["text"],
                metadata={"score": record["score"], "node_id": record["node_id"]}
            )
            for record in records_list
        ]
        
        return RawSearchResult(items=items, records=records_list)


class MultiHopRetriever(Retriever):
    """
    支援多跳推理的自定義檢索器
    
    檢索策略:
    1. 向量檢索初始 Chunks (0-hop)
    2. 擴展到 MENTIONS 的 Entities (1-hop)
    3. 沿著 RELATION 邊遍歷相鄰 Entities (2-hop, 3-hop, ...)
    4. 收集路徑上的所有 Chunks 作為上下文
    """
    
    def __init__(
        self,
        driver,
        embedder,
        hop_count: int = 1,
        top_k: int = 15,
        rerank_threshold: float = 0.6,
        vector_index_name: str = "chunk_embeddings",
        neo4j_database: str = None,
    ):
        self.driver = driver
        self.embedder = embedder
        self.hop_count = hop_count
        self.top_k = top_k
        self.rerank_threshold = rerank_threshold
        self.vector_index_name = vector_index_name
        self.neo4j_database = neo4j_database
        
    def search(
        self,
        query_text: str = None,
        query_vector: List[float] = None,
        top_k: int = None,
    ) -> RawSearchResult:
        """執行多跳檢索"""
        
        if top_k is None:
            top_k = self.top_k
        
        # 1. 獲取查詢向量
        if query_vector is None and query_text:
            query_vector = self.embedder.embed_query(query_text)
        
        # 2. 向量檢索初始 Chunks
        with self.driver.session(database=self.neo4j_database) as session:
            initial_result = session.run(f"""
                CALL db.index.vector.queryNodes(
                    $index_name, 
                    $top_k, 
                    $query_vector
                )
                YIELD node, score
                RETURN node.text as text, 
                       elementId(node) as node_id,
                       score
                ORDER BY score DESC
            """, {
                "index_name": self.vector_index_name,
                "top_k": top_k,
                "query_vector": query_vector
            })
            
            initial_records = list(initial_result)  # Neo4j Record objects
        
        # 3. 多跳擴展
        all_records = list(initial_records)
        collected_entities = set()
        
        for record in initial_records:
            # 獲取此 Chunk 提及的 Entities
            with self.driver.session(database=self.neo4j_database) as session:
                entities = session.run("""
                    MATCH (c:Chunk)
                    WHERE elementId(c) = $chunk_id
                    MATCH (c)-[:MENTIONS]->(e:__Entity__)
                    RETURN DISTINCT elementId(e) as entity_id, e.id as entity_name
                    LIMIT 20
                """, {
                    "chunk_id": record["node_id"]
                }).data()
                
                for entity in entities:
                    collected_entities.add(entity["entity_id"])
        
        # 4. 沿著關係邊遍歷（多跳）
        for hop in range(self.hop_count):
            if not collected_entities:
                break
                
            with self.driver.session(database=self.neo4j_database) as session:
                # 獲取相鄰實體
                neighbor_result = session.run("""
                    MATCH (e:__Entity__)
                    WHERE elementId(e) IN $entity_ids
                    MATCH (e)-[r]->(neighbor:__Entity__)
                    WHERE NOT elementId(neighbor) IN $entity_ids
                    WITH DISTINCT neighbor
                    LIMIT 30
                    MATCH (c:Chunk)-[:MENTIONS]->(neighbor)
                    RETURN DISTINCT c.text as text,
                           elementId(c) as node_id,
                           0.5 as score
                """, {
                    "entity_ids": list(collected_entities)
                })
                
                new_records = list(neighbor_result)
                all_records.extend(new_records)
        
        # 5. 限制總數量並構建返回結果
        limited_records = all_records[:top_k * 2]
        
        items = [
            RetrieverResultItem(
                content=record["text"],
                metadata={"score": record["score"], "node_id": record["node_id"]}
            )
            for record in limited_records
        ]
        
        return RawSearchResult(items=items, records=limited_records)

print("✅ 向量檢索器和多跳檢索器定義完成")

✅ 向量檢索器和多跳檢索器定義完成


In [24]:
# ============================================================
# 🧪 第 6 步：QA 生成與評估函數
# ============================================================

def generate_answer(
    question: str,
    retriever,
    llm,
    reference_answer: str = None
) -> Dict[str, Any]:
    """
    使用 Graph RAG 生成答案並評估
    
    Args:
        question: 問題文字
        retriever: 檢索器實例
        llm: 大型語言模型
        reference_answer: 參考答案（用於評估）
        
    Returns:
        包含答案、檢索上下文、評估指標的字典
    """
    start_time = time.time()
    
    # 1. 檢索相關上下文
    search_result = retriever.search(query_text=question, top_k=15)
    context_str = "\n\n".join([item.content for item in search_result.items])
    
    # 2. 生成答案
    prompt = GRAPH_RAG_SYSTEM_PROMPT.format(
        context_str=context_str,
        query_str=question
    )
    
    response = llm.invoke(prompt)
    
    latency = time.time() - start_time
    
    # 3. 評估指標
    metrics = {}
    if reference_answer:
        metrics["f1_score"] = calculate_f1_score(response.content, reference_answer)
        metrics["cosine_similarity"] = calculate_cosine_similarity(response.content, reference_answer)
    
    return {
        "question": question,
        "answer": response.content,
        "context": context_str,
        "retrieved_chunks": len(search_result.items),
        "latency": latency,
        "metrics": metrics
    }

def run_qa_batch(
    questions: List[Dict],
    retriever,
    llm,
    max_questions: int = None
) -> List[Dict[str, Any]]:
    """
    批次執行 QA
    
    Args:
        questions: 問題列表（每個問題是一個字典，包含 'question' 和可選的 'answer'）
        retriever: 檢索器實例
        llm: 大型語言模型
        max_questions: 最大問題數量限制
        
    Returns:
        結果列表
    """
    results = []
    question_subset = questions[:max_questions] if max_questions else questions
    
    for idx, q in enumerate(question_subset):
        print(f"[{idx+1}/{len(question_subset)}] 處理問題: {q['question'][:50]}...")
        
        result = generate_answer(
            question=q['question'],
            retriever=retriever,
            llm=llm,
            reference_answer=q.get('answer')
        )
        
        result['question_id'] = idx
        results.append(result)
    
    return results

print("✅ QA 生成與評估函數定義完成")

✅ QA 生成與評估函數定義完成


In [25]:
# ============================================================
# 🌟 第 7 步：Algorithm 1 - 星型拓樸偵測 (Star Topology Detection)
# ============================================================

def detect_hub_entities(driver, percentile: float = 95) -> List[str]:
    """
    Algorithm 1: 偵測星型拓樸的中心節點 (Hub Entities)
    
    依據論文 Section 3.5：
    - 計算每個實體的度數 (Degree)
    - 識別度數超過第 95 百分位的實體作為 Hub
    
    Args:
        driver: Neo4j 驅動器
        percentile: 百分位閾值
        
    Returns:
        Hub 實體的 ID 列表
    """
    with driver.session() as session:
        # 計算所有實體的度數
        result = session.run("""
            MATCH (e:__Entity__)
            OPTIONAL MATCH (e)-[r]-()
            WITH e, count(DISTINCT r) as degree
            RETURN e.id as entity_id, degree
            ORDER BY degree DESC
        """)
        
        degrees = [(r["entity_id"], r["degree"]) for r in result]
        
        if not degrees:
            return []
        
        # 計算百分位閾值
        degree_values = [d[1] for d in degrees]
        threshold = np.percentile(degree_values, percentile)
        
        # 篩選 Hub 實體
        hub_entities = [entity_id for entity_id, degree in degrees if degree >= threshold]
        
        print(f"✅ 偵測到 {len(hub_entities)} 個 Hub 實體 (度數閾值: {threshold:.2f})")
        
        return hub_entities

print("✅ Algorithm 1 定義完成")

✅ Algorithm 1 定義完成


In [26]:
# ============================================================
# 🔗 第 8 步：Algorithm 2 - 圖譜增強 (Graph Enhancement)
# ============================================================

def enhance_knowledge_graph(
    driver,
    llm,
    hub_entities: List[str],
    questions: List[Dict],
    max_iterations: int = 3,
    quality_threshold: float = 2.0
) -> Dict[str, Any]:
    """
    Algorithm 2: 知識圖譜增強
    
    依據論文 Section 3.5：
    1. 全域推論 (Global Inference): 連接孤立的 Hub 實體
    2. 問題導向密集化 (Question-Oriented Densification): 根據問題集增強關係
    
    Args:
        driver: Neo4j 驅動器
        llm: 大型語言模型（用於推論新關係）
        hub_entities: Hub 實體列表
        questions: 問題集（用於導向密集化）
        max_iterations: 最大迭代次數
        quality_threshold: 目標關係密度
        
    Returns:
        增強統計資訊
    """
    print("🔗 開始圖譜增強...")
    
    iteration = 0
    initial_metrics = calculate_graph_metrics(driver)
    print(f"初始關係密度: {initial_metrics['relation_density']:.2f}")
    
    while iteration < max_iterations:
        iteration += 1
        print(f"\n--- 迭代 {iteration}/{max_iterations} ---")
        
        # Step 1: 全域推論 - 連接孤立的 Hub 實體
        with driver.session() as session:
            # 找出孤立的 Hub 實體對
            isolated_pairs = session.run("""
                MATCH (h1:__Entity__)
                WHERE h1.id IN $hub_ids
                MATCH (h2:__Entity__)
                WHERE h2.id IN $hub_ids AND h1.id < h2.id
                WHERE NOT (h1)-[]-(h2)
                RETURN h1.id as entity1, h2.id as entity2
                LIMIT 10
            """, {"hub_ids": hub_entities}).data()
            
            print(f"發現 {len(isolated_pairs)} 對孤立的 Hub 實體")
            
            # 使用 LLM 推論可能的關係
            for pair in isolated_pairs:
                prompt = f"""
                Based on domain knowledge in Goat Disease:
                Entity 1: {pair['entity1']}
                Entity 2: {pair['entity2']}
                
                If there is a logical relation between these entities, 
                output the relation type (e.g., "causes", "treats", "prevents").
                If no relation exists, output "NONE".
                
                Relation Type:
                """
                
                try:
                    response = llm.invoke(prompt)
                    relation_type = response.content.strip()
                    
                    if relation_type and relation_type != "NONE":
                        # 建立新關係
                        session.run("""
                            MATCH (e1:__Entity__ {id: $entity1})
                            MATCH (e2:__Entity__ {id: $entity2})
                            MERGE (e1)-[r:INFERRED_RELATION]->(e2)
                            SET r.type = $relation_type, r.source = 'global_inference'
                        """, {
                            "entity1": pair['entity1'],
                            "entity2": pair['entity2'],
                            "relation_type": relation_type
                        })
                        print(f"  ✅ 建立關係: {pair['entity1']} -{relation_type}-> {pair['entity2']}")
                except Exception as e:
                    print(f"  ❌ 推論失敗: {e}")
        
        # Step 2: 問題導向密集化
        # 根據問題集中的關鍵詞，增強相關實體之間的連接
        with driver.session() as session:
            for q in questions[:20]:  # 使用前 20 個問題
                # 提取問題中的實體
                entities_in_question = session.run("""
                    MATCH (e:__Entity__)
                    WHERE toLower($question) CONTAINS toLower(e.id)
                    RETURN e.id as entity_id
                    LIMIT 5
                """, {"question": q['question']}).data()
                
                entity_ids = [e['entity_id'] for e in entities_in_question]
                
                if len(entity_ids) >= 2:
                    # 增強這些實體之間的連接
                    for i in range(len(entity_ids)):
                        for j in range(i+1, len(entity_ids)):
                            session.run("""
                                MATCH (e1:__Entity__ {id: $entity1})
                                MATCH (e2:__Entity__ {id: $entity2})
                                WHERE NOT (e1)-[]-(e2)
                                MERGE (e1)-[r:QUESTION_LINKED]->(e2)
                                SET r.source = 'question_oriented'
                            """, {
                                "entity1": entity_ids[i],
                                "entity2": entity_ids[j]
                            })
        
        # 檢查是否達到品質閾值
        current_metrics = calculate_graph_metrics(driver)
        print(f"當前關係密度: {current_metrics['relation_density']:.2f}")
        
        if current_metrics['relation_density'] >= quality_threshold:
            print(f"✅ 達到品質閾值 ({quality_threshold})")
            break
    
    final_metrics = calculate_graph_metrics(driver)
    
    return {
        "iterations": iteration,
        "initial_density": initial_metrics['relation_density'],
        "final_density": final_metrics['relation_density'],
        "initial_weak_ratio": initial_metrics['weak_entity_ratio'],
        "final_weak_ratio": final_metrics['weak_entity_ratio'],
        "improvement": final_metrics['relation_density'] - initial_metrics['relation_density']
    }

print("✅ Algorithm 2 定義完成")

✅ Algorithm 2 定義完成


# 🚀 主執行流程 (Main Execution Workflow)

---

## 準備工作

在執行實驗前，請確保：
1. ✅ Neo4j 資料庫已啟動
2. ✅ Ollama 服務已啟動
3. ✅ 知識庫檔案 `goat_data_text collection-1.2-eng.txt` 存在
4. ✅ 問題集檔案 `topic.csv` 存在

---

## 執行順序

請依序執行以下 4 個階段的 Cell：
1. **Phase 1**: 索引優化 (Indexing Optimization)
2. **Phase 2**: 基準線與診斷 (Baseline & Diagnosis)
3. **Phase 3**: 結構優化 (Structural Optimization)
4. **Phase 4**: 檢索消融 (Retrieval Ablation)

In [ ]:
# ============================================================
# 📊 Phase 1: 索引優化 (Indexing Optimization)
# ============================================================

print("=" * 60)
print("Phase 1: 索引優化消融實驗")
print("=" * 60)

# 初始化連接
driver = GraphDatabase.driver(
    CONFIG['infrastructure']['neo4j_uri'],
    auth=CONFIG['infrastructure']['neo4j_auth']
)

# 🔥 使用自定義的 Ollama Wrapper
embedder = OllamaEmbedder(model=CONFIG['models']['embed_model'])

llm = OllamaLLMWrapper(
    model_name=CONFIG['models']['llm_model'],
    temperature=CONFIG['generation']['temperature']
)

# 載入知識庫
print("\n📂 載入資料...")
with open(KNOWLEDGE_BASE_PATH, 'r', encoding='utf-8') as f:
    knowledge_text = f.read()

# 載入問題集
questions_df = pd.read_csv(QUESTION_DATASET_PATH)
questions = questions_df.to_dict('records')

print(f"✅ 載入 {len(knowledge_text)} 個字元的知識庫")
print(f"✅ 載入 {len(questions)} 個問題")

# 執行消融實驗
phase1_results = []

# 🎯 Phase 1 & 2 的固定檢索參數（控制變因）
FIXED_RETRIEVAL_PARAMS = {
    "hop_count": 2,           # 固定 2 跳 - Graph RAG 的標準深度
    "top_k": 15,              # 固定 Top-K=15 - 用戶預設值
    "rerank_threshold": 0.60  # 固定門檻 0.6 - 中等嚴格度
}

print(f"\n🔒 Phase 1 固定檢索參數: Hop={FIXED_RETRIEVAL_PARAMS['hop_count']}, Top-K={FIXED_RETRIEVAL_PARAMS['top_k']}, Threshold={FIXED_RETRIEVAL_PARAMS['rerank_threshold']}")

for idx, config in enumerate(CONFIG['indexing_grid']):
    print(f"\n{'='*60}")
    print(f"[{idx+1}/{len(CONFIG['indexing_grid'])}] 測試配置: Chunk={config['chunk_size']}, Overlap={config['overlap']}")
    print(f"{'='*60}")
    
    try:
        # 🔥 使用自定義 Prompt 建構圖譜
        build_stats = build_knowledge_graph_with_custom_prompt(
            text_content=knowledge_text,
            chunk_size=config['chunk_size'],
            chunk_overlap=config['overlap'],
            driver=driver,
            embedder=embedder,
            llm=llm
        )
        
        print(f"\n  ✅ 建構完成: {build_stats['chunk_count']} chunks, {build_stats['entity_count']} entities, {build_stats['relation_count']} relations")
        
        # 計算圖譜品質
        metrics = calculate_graph_metrics(driver)
        print(f"  📊 圖譜品質: Density={metrics['relation_density']:.3f}, Weak Entity Ratio={metrics['weak_entity_ratio']:.2%}")
        
        # 🧪 執行 QA 評估（使用固定檢索參數）
        print(f"\n  🔍 執行 QA 評估 (限制 {CONFIG['generation']['max_questions']} 個問題)...")
        retriever = MultiHopRetriever(
            driver=driver,
            embedder=embedder,
            hop_count=FIXED_RETRIEVAL_PARAMS['hop_count'],
            top_k=FIXED_RETRIEVAL_PARAMS['top_k'],
            rerank_threshold=FIXED_RETRIEVAL_PARAMS['rerank_threshold']
        )
        
        qa_results = run_qa_batch(
            questions=questions,
            retriever=retriever,
            llm=llm,
            max_questions=CONFIG['generation']['max_questions']
        )
        
        # 計算平均指標
        valid_results = [r for r in qa_results if r['metrics']]
        if valid_results:
            avg_f1 = np.mean([r['metrics']['f1_score'] for r in valid_results])
            avg_cosine = np.mean([r['metrics']['cosine_similarity'] for r in valid_results])
            avg_latency = np.mean([r['latency'] for r in valid_results])
        else:
            avg_f1 = avg_cosine = avg_latency = 0.0
        
        print(f"  ✅ QA 平均指標: F1={avg_f1:.3f}, Cosine={avg_cosine:.3f}, Latency={avg_latency:.2f}s")
        
        # 記錄結果
        result_row = {
            "phase": "indexing_optimization",
            "chunk_size": config['chunk_size'],
            "overlap": config['overlap'],
            "hop_count": FIXED_RETRIEVAL_PARAMS['hop_count'],
            "top_k": FIXED_RETRIEVAL_PARAMS['top_k'],
            "rerank_threshold": FIXED_RETRIEVAL_PARAMS['rerank_threshold'],
            "chunk_count": build_stats['chunk_count'],
            "entity_count": build_stats['entity_count'],
            "relation_count": build_stats['relation_count'],
            "relation_density": metrics['relation_density'],
            "weak_entity_ratio": metrics['weak_entity_ratio'],
            "avg_f1": avg_f1,
            "avg_cosine_sim": avg_cosine,
            "avg_latency": avg_latency,
            "build_time": build_stats['build_time']
        }
        
        phase1_results.append(result_row)
        save_csv_metric(result_row, "ablation_chunking.csv")
        
        # 儲存詳細的 QA 日誌
        for qa_result in qa_results:
            log_entry = {
                "phase": "indexing_optimization",
                "chunk_size": config['chunk_size'],
                "overlap": config['overlap'],
                **qa_result
            }
            save_jsonl_log(log_entry, "phase1_qa_details.jsonl")
        
        print(f"  💾 結果已儲存")
        
    except Exception as e:
        print(f"  ❌ 配置失敗: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        clear_memory()

print("\n" + "=" * 60)
print("Phase 1 完成！結果已儲存至 ablation_chunking.csv 和 phase1_qa_details.jsonl")
print("=" * 60)

Phase 1: 索引優化消融實驗

📂 載入資料...
✅ 載入 694140 個字元的知識庫
✅ 載入 15 個問題

🔒 Phase 1 固定檢索參數: Hop=2, Top-K=15, Threshold=0.6

[1/1] 測試配置: Chunk=2048, Overlap=512
🗑️ 清空現有圖譜...
✂️ 切塊中 (size=2048, overlap=512)...
   生成 452 個 chunks
🔍 使用高密度 Prompt 抽取三元組...
   處理進度: 10/452 (成功: 9, 失敗: 0)
   處理進度: 20/452 (成功: 19, 失敗: 0)
   處理進度: 30/452 (成功: 29, 失敗: 0)


In [17]:
# ============================================================
# 📈 Phase 2: 基準線與診斷 (Baseline & Diagnosis)
# ============================================================

print("=" * 60)
print("Phase 2: 基準線與診斷 - Before State")
print("=" * 60)

# 🎯 Phase 2 同樣使用固定檢索參數（控制變因）
FIXED_RETRIEVAL_PARAMS = {
    "hop_count": 2,           # 固定 2 跳
    "top_k": 15,              # 固定 Top-K=15
    "rerank_threshold": 0.60  # 固定門檻 0.6
}

print(f"🔒 Phase 2 固定檢索參數: Hop={FIXED_RETRIEVAL_PARAMS['hop_count']}, Top-K={FIXED_RETRIEVAL_PARAMS['top_k']}, Threshold={FIXED_RETRIEVAL_PARAMS['rerank_threshold']}")

try:
    # 使用最佳索引配置建構圖譜
    optimal_config = CONFIG['optimal_indexing']
    print(f"\n使用最佳配置: Chunk={optimal_config['chunk_size']}, Overlap={optimal_config['overlap']}")

    build_stats = build_knowledge_graph_with_custom_prompt(
        text_content=knowledge_text,
        chunk_size=optimal_config['chunk_size'],
        chunk_overlap=optimal_config['overlap'],
        driver=driver,
        embedder=embedder,
        llm=llm
    )

    # 計算初始圖譜品質
    initial_metrics = calculate_graph_metrics(driver)
    print(f"\n📊 初始圖譜品質指標:")
    print(f"  - 關係密度 (Relation Density): {initial_metrics['relation_density']:.3f}")
    print(f"  - 弱實體比例 (Weak Entity Ratio): {initial_metrics['weak_entity_ratio']:.2%}")

    # 測試 1: Vector-Only RAG (基準線)
    print("\n🔍 測試 1: Vector-Only RAG (純向量檢索)...")
    vector_retriever = VectorRetriever(
        driver=driver,
        index_name=CONFIG['infrastructure']['vector_index_name'],
        embedder=embedder
    )

    vector_results = run_qa_batch(
        questions=questions,
        retriever=vector_retriever,
        llm=llm,
        max_questions=10
    )

    valid_vector = [r for r in vector_results if r['metrics']]
    vector_f1 = np.mean([r['metrics']['f1_score'] for r in valid_vector]) if valid_vector else 0.0
    vector_cosine = np.mean([r['metrics']['cosine_similarity'] for r in valid_vector]) if valid_vector else 0.0
    vector_latency = np.mean([r['latency'] for r in vector_results]) if vector_results else 0.0

    print(f"  ✅ Vector-Only: F1={vector_f1:.3f}, Cosine={vector_cosine:.3f}, Latency={vector_latency:.2f}s")
    
    clear_memory()

    # 測試 2: Initial Graph RAG（使用固定檢索參數）
    print(f"\n🔍 測試 2: Initial Graph RAG (Hop={FIXED_RETRIEVAL_PARAMS['hop_count']}, Top-K={FIXED_RETRIEVAL_PARAMS['top_k']})...")
    graph_retriever = MultiHopRetriever(
        driver=driver,
        embedder=embedder,
        hop_count=FIXED_RETRIEVAL_PARAMS['hop_count'],
        top_k=FIXED_RETRIEVAL_PARAMS['top_k'],
        rerank_threshold=FIXED_RETRIEVAL_PARAMS['rerank_threshold']
    )

    graph_results = run_qa_batch(
        questions=questions,
        retriever=graph_retriever,
        llm=llm,
        max_questions=10
    )

    valid_graph = [r for r in graph_results if r['metrics']]
    graph_f1 = np.mean([r['metrics']['f1_score'] for r in valid_graph]) if valid_graph else 0.0
    graph_cosine = np.mean([r['metrics']['cosine_similarity'] for r in valid_graph]) if valid_graph else 0.0
    graph_latency = np.mean([r['latency'] for r in graph_results]) if graph_results else 0.0

    print(f"  ✅ Graph RAG: F1={graph_f1:.3f}, Cosine={graph_cosine:.3f}, Latency={graph_latency:.2f}s")

    # 記錄基準線結果
    baseline_results = [
        {
            "phase": "baseline",
            "method": "Vector-Only RAG",
            "hop_count": 0,
            "top_k": FIXED_RETRIEVAL_PARAMS['top_k'],
            "rerank_threshold": 0.0,
            "avg_f1": vector_f1,
            "avg_cosine_sim": vector_cosine,
            "avg_latency": vector_latency,
            "relation_density": initial_metrics['relation_density'],
            "weak_entity_ratio": initial_metrics['weak_entity_ratio']
        },
        {
            "phase": "baseline",
            "method": "Initial Graph RAG",
            "hop_count": FIXED_RETRIEVAL_PARAMS['hop_count'],
            "top_k": FIXED_RETRIEVAL_PARAMS['top_k'],
            "rerank_threshold": FIXED_RETRIEVAL_PARAMS['rerank_threshold'],
            "avg_f1": graph_f1,
            "avg_cosine_sim": graph_cosine,
            "avg_latency": graph_latency,
            "relation_density": initial_metrics['relation_density'],
            "weak_entity_ratio": initial_metrics['weak_entity_ratio']
        }
    ]

    for result in baseline_results:
        save_csv_metric(result, "baseline_comparison.csv")

    # 儲存詳細 QA 日誌
    for result in vector_results:
        log_entry = {"phase": "baseline", "method": "Vector-Only", **result}
        save_jsonl_log(log_entry, "phase2_qa_details.jsonl")
    
    for result in graph_results:
        log_entry = {"phase": "baseline", "method": "Graph RAG", **result}
        save_jsonl_log(log_entry, "phase2_qa_details.jsonl")

    print("\n" + "=" * 60)
    print("Phase 2 完成！結果已儲存至 baseline_comparison.csv 和 phase2_qa_details.jsonl")
    print("=" * 60)

except Exception as e:
    print(f"❌ Phase 2 執行失敗: {e}")
    import traceback
    traceback.print_exc()

finally:
    clear_memory()

Phase 2: 基準線與診斷 - Before State
🔒 Phase 2 固定檢索參數: Hop=2, Top-K=15, Threshold=0.6

使用最佳配置: Chunk=2048, Overlap=512
🗑️ 清空現有圖譜...
✂️ 切塊中 (size=2048, overlap=512)...
   生成 452 個 chunks
🔍 使用高密度 Prompt 抽取三元組...
   處理進度: 10/452 (成功: 9, 失敗: 0)
   處理進度: 20/452 (成功: 19, 失敗: 0)


KeyboardInterrupt: 

In [ ]:
# ============================================================
# 🔧 Phase 3: 結構優化 (Structural Optimization)
# ============================================================

print("=" * 60)
print("Phase 3: 結構優化 - Graph Enhancement")
print("=" * 60)

# 🎯 Phase 3 同樣使用固定檢索參數（控制變因）
FIXED_RETRIEVAL_PARAMS = {
    "hop_count": 2,
    "top_k": 15,
    "rerank_threshold": 0.60
}

print(f"🔒 Phase 3 固定檢索參數: Hop={FIXED_RETRIEVAL_PARAMS['hop_count']}, Top-K={FIXED_RETRIEVAL_PARAMS['top_k']}, Threshold={FIXED_RETRIEVAL_PARAMS['rerank_threshold']}")

try:
    # Step 1: 記錄優化前的狀態
    print("\n📸 記錄優化前的基準表現...")
    retriever_before = MultiHopRetriever(
        driver=driver,
        embedder=embedder,
        hop_count=FIXED_RETRIEVAL_PARAMS['hop_count'],
        top_k=FIXED_RETRIEVAL_PARAMS['top_k'],
        rerank_threshold=FIXED_RETRIEVAL_PARAMS['rerank_threshold']
    )
    
    results_before = run_qa_batch(
        questions=questions,
        retriever=retriever_before,
        llm=llm,
        max_questions=10
    )
    
    valid_before = [r for r in results_before if r['metrics']]
    f1_before = np.mean([r['metrics']['f1_score'] for r in valid_before]) if valid_before else 0.0
    cosine_before = np.mean([r['metrics']['cosine_similarity'] for r in valid_before]) if valid_before else 0.0
    
    print(f"  ✅ 優化前: F1={f1_before:.3f}, Cosine={cosine_before:.3f}")
    
    # Step 2: 偵測 Hub 實體
    print("\n🌟 執行 Algorithm 1: 星型拓樸偵測...")
    hub_entities = detect_hub_entities(
        driver=driver,
        percentile=CONFIG['optimization']['hub_threshold_percentile']
    )

    print(f"  發現 {len(hub_entities)} 個 Hub 實體")
    if hub_entities:
        print(f"  Hub 實體範例:")
        for hub in hub_entities[:10]:
            print(f"    - {hub}")

    # Step 3: 圖譜增強
    print("\n🔗 執行 Algorithm 2: 圖譜增強...")
    enhancement_stats = enhance_knowledge_graph(
        driver=driver,
        llm=llm,
        hub_entities=hub_entities,
        questions=questions,
        max_iterations=CONFIG['optimization']['max_iterations'],
        quality_threshold=CONFIG['optimization']['quality_threshold']
    )

    print(f"\n📊 增強結果:")
    print(f"  - 迭代次數: {enhancement_stats['iterations']}")
    print(f"  - 初始密度: {enhancement_stats['initial_density']:.3f}")
    print(f"  - 最終密度: {enhancement_stats['final_density']:.3f}")
    print(f"  - 提升幅度: {enhancement_stats['improvement']:.3f}")
    print(f"  - 初始弱實體比例: {enhancement_stats['initial_weak_ratio']:.2%}")
    print(f"  - 最終弱實體比例: {enhancement_stats['final_weak_ratio']:.2%}")

    # Step 4: 評估優化後的效果
    print("\n🧪 評估優化後的 QA 表現...")
    retriever_after = MultiHopRetriever(
        driver=driver,
        embedder=embedder,
        hop_count=FIXED_RETRIEVAL_PARAMS['hop_count'],
        top_k=FIXED_RETRIEVAL_PARAMS['top_k'],
        rerank_threshold=FIXED_RETRIEVAL_PARAMS['rerank_threshold']
    )
    
    results_after = run_qa_batch(
        questions=questions,
        retriever=retriever_after,
        llm=llm,
        max_questions=10
    )
    
    valid_after = [r for r in results_after if r['metrics']]
    f1_after = np.mean([r['metrics']['f1_score'] for r in valid_after]) if valid_after else 0.0
    cosine_after = np.mean([r['metrics']['cosine_similarity'] for r in valid_after]) if valid_after else 0.0
    
    print(f"  ✅ 優化後: F1={f1_after:.3f}, Cosine={cosine_after:.3f}")
    print(f"  📈 改善: ΔF1={f1_after-f1_before:+.3f}, ΔCosine={cosine_after-cosine_before:+.3f}")

    # 記錄優化結果
    optimization_result = {
        "phase": "optimization",
        "hop_count": FIXED_RETRIEVAL_PARAMS['hop_count'],
        "top_k": FIXED_RETRIEVAL_PARAMS['top_k'],
        "rerank_threshold": FIXED_RETRIEVAL_PARAMS['rerank_threshold'],
        "iterations": enhancement_stats['iterations'],
        "initial_density": enhancement_stats['initial_density'],
        "final_density": enhancement_stats['final_density'],
        "initial_weak_ratio": enhancement_stats['initial_weak_ratio'],
        "final_weak_ratio": enhancement_stats['final_weak_ratio'],
        "improvement": enhancement_stats['improvement'],
        "hub_count": len(hub_entities),
        "f1_before": f1_before,
        "f1_after": f1_after,
        "f1_improvement": f1_after - f1_before,
        "cosine_before": cosine_before,
        "cosine_after": cosine_after,
        "cosine_improvement": cosine_after - cosine_before
    }

    save_csv_metric(optimization_result, "optimization_results.csv")
    
    # 儲存詳細 QA 日誌
    for result in results_after:
        log_entry = {"phase": "optimization_after", **result}
        save_jsonl_log(log_entry, "phase3_qa_details.jsonl")

    print("\n" + "=" * 60)
    print("Phase 3 完成！結果已儲存至 optimization_results.csv 和 phase3_qa_details.jsonl")
    print("=" * 60)

except Exception as e:
    print(f"❌ Phase 3 執行失敗: {e}")
    import traceback
    traceback.print_exc()

finally:
    clear_memory()

In [ ]:
# ============================================================
# 🎯 Phase 4: 檢索消融 (Retrieval Ablation)
# ============================================================

print("=" * 60)
print("Phase 4: 檢索消融實驗 - After State")
print("=" * 60)

# 🔥 Phase 4 是唯一會測試不同檢索參數的階段
print("\n⚠️ Phase 4 開始測試不同的 Hop Count 和 Top-K 組合")

try:
    # 使用優化後的圖譜進行檢索消融實驗
    retrieval_grid = CONFIG['retrieval_grid']

    phase4_results = []
    jsonl_filename = "final_experiment.jsonl"

    # 清空 JSONL 檔案
    if Path(jsonl_filename).exists():
        Path(jsonl_filename).unlink()

    total_combinations = len(retrieval_grid['hop_counts']) * len(retrieval_grid['top_k_values'])
    print(f"\n開始測試 {len(retrieval_grid['hop_counts'])} × {len(retrieval_grid['top_k_values'])} = {total_combinations} 種組合...")
    print(f"每種組合測試 {min(CONFIG['max_questions'], len(questions))} 個問題\n")

    combination_idx = 0
    for hop_count in retrieval_grid['hop_counts']:
        for top_k in retrieval_grid['top_k_values']:
            combination_idx += 1
            print(f"\n{'='*60}")
            print(f"[{combination_idx}/{total_combinations}] 測試配置: Hop={hop_count}, TopK={top_k}")
            print(f"{'='*60}")
            
            try:
                # 建立檢索器
                retriever = MultiHopRetriever(
                    driver=driver,
                    embedder=embedder,
                    hop_count=hop_count,
                    top_k=top_k,
                    rerank_threshold=CONFIG['generation']['rerank_threshold']
                )
                
                # 執行 QA 批次
                batch_results = []
                max_questions = min(CONFIG['max_questions'], len(questions))
                
                for idx, q in enumerate(questions[:max_questions]):
                    if (idx + 1) % 5 == 0:
                        print(f"  處理進度: {idx+1}/{max_questions}")
                        clear_memory()
                    
                    result = generate_answer(
                        question=q['question'],
                        retriever=retriever,
                        llm=llm,
                        reference_answer=q.get('answer')
                    )
                    
                    # 記錄詳細日誌至 JSONL
                    log_record = {
                        "question_id": idx,
                        "question_text": q['question'],
                        "reference_answer": q.get('answer', ''),
                        "model_response": result['answer'],
                        "retrieved_chunks": result['retrieved_chunks'],
                        "context_preview": result['context'][:500] + "..." if len(result['context']) > 500 else result['context'],
                        "metrics": result['metrics'],
                        "latency": result['latency'],
                        "config_snapshot": {
                            "hop_count": hop_count,
                            "top_k": top_k,
                            "rerank_threshold": CONFIG['generation']['rerank_threshold'],
                            "chunk_size": CONFIG['optimal_indexing']['chunk_size'],
                            "overlap": CONFIG['optimal_indexing']['overlap']
                        },
                        "timestamp": datetime.now().isoformat()
                    }
                    
                    save_jsonl_log(log_record, jsonl_filename)
                    batch_results.append(result)
                
                # 計算批次統計指標
                valid_results = [r for r in batch_results if r['metrics']]
                avg_f1 = np.mean([r['metrics']['f1_score'] for r in valid_results]) if valid_results else 0.0
                avg_cosine = np.mean([r['metrics']['cosine_similarity'] for r in valid_results]) if valid_results else 0.0
                avg_latency = np.mean([r['latency'] for r in batch_results]) if batch_results else 0.0
                
                # 獲取當前圖譜品質
                current_metrics = calculate_graph_metrics(driver)
                
                # 記錄至 CSV
                result_row = {
                    "phase": "retrieval_ablation",
                    "hop_count": hop_count,
                    "top_k": top_k,
                    "rerank_threshold": CONFIG['generation']['rerank_threshold'],
                    "avg_f1": avg_f1,
                    "avg_cosine_sim": avg_cosine,
                    "avg_latency": avg_latency,
                    "relation_density": current_metrics['relation_density'],
                    "weak_entity_ratio": current_metrics['weak_entity_ratio'],
                    "questions_tested": len(batch_results)
                }
                
                phase4_results.append(result_row)
                save_csv_metric(result_row, "retrieval_ablation.csv")
                
                print(f"\n  ✅ 結果: F1={avg_f1:.3f}, Cosine={avg_cosine:.3f}, Latency={avg_latency:.2f}s")
            
            except Exception as e:
                print(f"  ❌ 配置 (Hop={hop_count}, TopK={top_k}) 失敗: {e}")
                import traceback
                traceback.print_exc()
            
            finally:
                clear_memory()

    print("\n" + "=" * 60)
    print("Phase 4 完成！")
    print(f"  - 詳細日誌: {jsonl_filename}")
    print(f"  - 統計表格: retrieval_ablation.csv")
    print(f"  - 測試組合數: {len(phase4_results)}/{total_combinations}")
    print("=" * 60)

except Exception as e:
    print(f"❌ Phase 4 執行失敗: {e}")
    import traceback
    traceback.print_exc()

finally:
    # 關閉連接
    driver.close()
    clear_memory()
    print("\n✅ 所有實驗完成！")

# 📊 結果分析與視覺化

以下 Cell 提供實驗結果的分析與視覺化功能。

In [ ]:
# ============================================================
# 📈 結果視覺化：Phase 1 索引消融實驗
# ============================================================

import matplotlib.pyplot as plt
import seaborn as sns

# 設定繁體中文字型
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 載入 Phase 1 結果
df_phase1 = pd.read_csv("ablation_chunking.csv")

# 建立視覺化
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. F1 Score vs Chunk Size
axes[0, 0].plot(df_phase1['chunk_size'], df_phase1['avg_f1'], marker='o', linewidth=2)
axes[0, 0].set_xlabel('Chunk Size', fontsize=12)
axes[0, 0].set_ylabel('Average F1 Score', fontsize=12)
axes[0, 0].set_title('Phase 1: F1 Score vs Chunk Size', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# 2. Cosine Similarity vs Chunk Size
axes[0, 1].plot(df_phase1['chunk_size'], df_phase1['avg_cosine_sim'], marker='s', color='orange', linewidth=2)
axes[0, 1].set_xlabel('Chunk Size', fontsize=12)
axes[0, 1].set_ylabel('Average Cosine Similarity', fontsize=12)
axes[0, 1].set_title('Phase 1: Cosine Similarity vs Chunk Size', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# 3. Relation Density vs Chunk Size
axes[1, 0].plot(df_phase1['chunk_size'], df_phase1['relation_density'], marker='^', color='green', linewidth=2)
axes[1, 0].set_xlabel('Chunk Size', fontsize=12)
axes[1, 0].set_ylabel('Relation Density', fontsize=12)
axes[1, 0].set_title('Phase 1: Relation Density vs Chunk Size', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Latency vs Chunk Size
axes[1, 1].plot(df_phase1['chunk_size'], df_phase1['avg_latency'], marker='d', color='red', linewidth=2)
axes[1, 1].set_xlabel('Chunk Size', fontsize=12)
axes[1, 1].set_ylabel('Average Latency (s)', fontsize=12)
axes[1, 1].set_title('Phase 1: Latency vs Chunk Size', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('phase1_ablation_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Phase 1 視覺化完成！圖片已儲存為 phase1_ablation_results.png")

In [ ]:
# ============================================================
# 📊 結果視覺化：Phase 4 檢索消融實驗（熱力圖）
# ============================================================

# 載入 Phase 4 結果
df_phase4 = pd.read_csv("retrieval_ablation.csv")

# 建立 F1 Score 的樞紐表
pivot_f1 = df_phase4.pivot(index='hop_count', columns='top_k', values='avg_f1')

# 建立 Cosine Similarity 的樞紐表
pivot_cosine = df_phase4.pivot(index='hop_count', columns='top_k', values='avg_cosine_sim')

# 建立視覺化
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. F1 Score 熱力圖
sns.heatmap(pivot_f1, annot=True, fmt='.3f', cmap='YlGnBu', ax=axes[0], cbar_kws={'label': 'F1 Score'})
axes[0].set_xlabel('Top-K', fontsize=12)
axes[0].set_ylabel('Hop Count', fontsize=12)
axes[0].set_title('Phase 4: F1 Score Heatmap (Hop Count × Top-K)', fontsize=14, fontweight='bold')

# 2. Cosine Similarity 熱力圖
sns.heatmap(pivot_cosine, annot=True, fmt='.3f', cmap='YlOrRd', ax=axes[1], cbar_kws={'label': 'Cosine Similarity'})
axes[1].set_xlabel('Top-K', fontsize=12)
axes[1].set_ylabel('Hop Count', fontsize=12)
axes[1].set_title('Phase 4: Cosine Similarity Heatmap (Hop Count × Top-K)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('phase4_retrieval_ablation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Phase 4 視覺化完成！圖片已儲存為 phase4_retrieval_ablation_heatmap.png")

In [ ]:
# ============================================================
# 📋 結果摘要表格生成
# ============================================================

print("=" * 80)
print("實驗結果摘要 (Experimental Results Summary)")
print("=" * 80)

# Phase 1: 索引優化最佳配置
df_phase1 = pd.read_csv("ablation_chunking.csv")
best_phase1 = df_phase1.loc[df_phase1['avg_f1'].idxmax()]

print("\n📊 Phase 1: 索引優化最佳配置")
print(f"  - Chunk Size: {int(best_phase1['chunk_size'])}")
print(f"  - Overlap: {int(best_phase1['overlap'])}")
print(f"  - F1 Score: {best_phase1['avg_f1']:.3f}")
print(f"  - Cosine Similarity: {best_phase1['avg_cosine_sim']:.3f}")
print(f"  - Relation Density: {best_phase1['relation_density']:.2f}")

# Phase 2: 基準線比較
try:
    df_phase2 = pd.read_csv("baseline_comparison.csv")
    print("\n📈 Phase 2: 基準線比較")
    for _, row in df_phase2.iterrows():
        print(f"  - {row['method']}:")
        print(f"      F1 Score: {row['avg_f1']:.3f}")
        print(f"      Cosine Similarity: {row['avg_cosine_sim']:.3f}")
except:
    print("\n⚠️ Phase 2 結果檔案不存在")

# Phase 3: 圖譜優化
try:
    df_phase3 = pd.read_csv("optimization_results.csv")
    opt_result = df_phase3.iloc[-1]
    print("\n🔧 Phase 3: 圖譜優化結果")
    print(f"  - 初始關係密度: {opt_result['initial_density']:.2f}")
    print(f"  - 最終關係密度: {opt_result['final_density']:.2f}")
    print(f"  - 提升幅度: {opt_result['improvement']:.2f}")
    print(f"  - Hub 實體數量: {int(opt_result['hub_count'])}")
except:
    print("\n⚠️ Phase 3 結果檔案不存在")

# Phase 4: 檢索消融最佳配置
df_phase4 = pd.read_csv("retrieval_ablation.csv")
best_phase4 = df_phase4.loc[df_phase4['avg_f1'].idxmax()]

print("\n🎯 Phase 4: 檢索消融最佳配置")
print(f"  - Hop Count: {int(best_phase4['hop_count'])}")
print(f"  - Top-K: {int(best_phase4['top_k'])}")
print(f"  - F1 Score: {best_phase4['avg_f1']:.3f}")
print(f"  - Cosine Similarity: {best_phase4['avg_cosine_sim']:.3f}")
print(f"  - Average Latency: {best_phase4['avg_latency']:.2f}s")

print("\n" + "=" * 80)
print("✅ 所有實驗結果摘要完成！")
print("=" * 80)

# 生成 LaTeX 表格（用於論文）
print("\n📝 LaTeX 表格（用於論文 Section 4.5）：")
print("\n% Table 1: Chunking Strategy Ablation Study")
print("\\begin{table}[h]")
print("\\centering")
print("\\caption{Impact of Chunking Strategies on System Performance}")
print("\\begin{tabular}{|c|c|c|c|c|}")
print("\\hline")
print("Chunk Size & Overlap & F1 Score & Cosine Sim & Relation Density \\\\")
print("\\hline")
for _, row in df_phase1.iterrows():
    print(f"{int(row['chunk_size'])} & {int(row['overlap'])} & {row['avg_f1']:.3f} & {row['avg_cosine_sim']:.3f} & {row['relation_density']:.2f} \\\\")
print("\\hline")
print("\\end{tabular}")
print("\\end{table}")

In [ ]:
# ============================================================
# 🔍 案例研究 (Case Study) - 從 JSONL 中抽取範例
# ============================================================

print("=" * 80)
print("案例研究 (Case Study) - Section 4.6")
print("=" * 80)

# 讀取 JSONL 日誌
jsonl_path = "final_experiment.jsonl"

if Path(jsonl_path).exists():
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        logs = [json.loads(line) for line in f]
    
    # 找出 F1 Score 最高的 3 個案例
    logs_sorted = sorted(logs, key=lambda x: x['metrics'].get('f1_score', 0), reverse=True)
    
    print("\n✅ F1 Score 最高的 3 個案例：\n")
    
    for idx, log in enumerate(logs_sorted[:3]):
        print(f"{'='*80}")
        print(f"案例 {idx+1}:")
        print(f"{'='*80}")
        print(f"\n📝 問題: {log['question_text']}")
        print(f"\n📌 參考答案: {log['reference_answer']}")
        print(f"\n🤖 模型回應: {log['model_response']}")
        print(f"\n📊 指標:")
        print(f"  - F1 Score: {log['metrics'].get('f1_score', 0):.3f}")
        print(f"  - Cosine Similarity: {log['metrics'].get('cosine_similarity', 0):.3f}")
        print(f"  - Latency: {log['latency']:.2f}s")
        print(f"\n🔍 檢索配置:")
        print(f"  - Hop Count: {log['config_snapshot']['hop_count']}")
        print(f"  - Top-K: {log['config_snapshot']['top_k']}")
        print(f"\n📄 檢索上下文預覽:")
        print(f"  {log['context_preview']}")
        print()
    
    # 找出 F1 Score 最低的 3 個案例（用於分析失敗原因）
    print("\n\n" + "="*80)
    print("⚠️ F1 Score 最低的 3 個案例（失敗分析）：\n")
    
    logs_worst = sorted(logs, key=lambda x: x['metrics'].get('f1_score', 0))
    
    for idx, log in enumerate(logs_worst[:3]):
        print(f"{'='*80}")
        print(f"失敗案例 {idx+1}:")
        print(f"{'='*80}")
        print(f"\n📝 問題: {log['question_text']}")
        print(f"\n📌 參考答案: {log['reference_answer']}")
        print(f"\n🤖 模型回應: {log['model_response']}")
        print(f"\n📊 指標:")
        print(f"  - F1 Score: {log['metrics'].get('f1_score', 0):.3f}")
        print(f"  - Cosine Similarity: {log['metrics'].get('cosine_similarity', 0):.3f}")
        print()
else:
    print("⚠️ JSONL 日誌檔案不存在，請先執行 Phase 4")

print("\n" + "=" * 80)
print("✅ 案例研究完成！")
print("=" * 80)

# 📚 使用說明與注意事項

---

## 🎯 執行順序

1. **依序執行上方所有 Cell**，從配置設定到 Phase 4
2. **不要跳過任何 Cell**，因為後續步驟依賴前面的定義
3. **執行時間預估**：
   - Phase 1: 約 30-60 分鐘（7 種配置）
   - Phase 2: 約 10-15 分鐘
   - Phase 3: 約 5-10 分鐘
   - Phase 4: 約 1-3 小時（取決於問題數量）

---

## 📁 輸出檔案

執行完成後會產生以下檔案：

### CSV 檔案（統計表格）
- `ablation_chunking.csv` - Phase 1 索引消融實驗結果
- `baseline_comparison.csv` - Phase 2 基準線比較
- `optimization_results.csv` - Phase 3 圖譜優化結果
- `retrieval_ablation.csv` - Phase 4 檢索消融實驗結果

### JSONL 檔案（詳細日誌）
- `final_experiment.jsonl` - Phase 4 每個問題的完整軌跡

### 視覺化圖片
- `phase1_ablation_results.png` - Phase 1 結果圖表
- `phase4_retrieval_ablation_heatmap.png` - Phase 4 熱力圖

---

## ⚙️ 配置調整

如需調整實驗參數，請修改第一個 Cell 中的 `CONFIG` 字典：

- **測試更多問題**: 增加 `CONFIG['generation']['max_questions']`
- **調整檢索深度**: 修改 `CONFIG['retrieval_grid']['hop_counts']`
- **更改 Top-K 值**: 修改 `CONFIG['retrieval_grid']['top_k_values']`
- **調整圖譜優化目標**: 修改 `CONFIG['optimization']['quality_threshold']`

---

## 🐛 故障排除

### 問題：Neo4j 連線失敗
**解決方法**: 檢查 Neo4j 是否啟動，並確認 URI 和認證資訊正確

### 問題：Ollama 模型載入失敗
**解決方法**: 執行 `ollama pull deepseek-r1:8b-llama-distill-q4_K_M` 和 `ollama pull nomic-embed-text:nomic`

### 問題：記憶體不足
**解決方法**: 減少 `max_questions` 或使用更小的 Chunk Size

---

## 📖 對應論文章節

- **Phase 1** → Section 4.5, Table 1
- **Phase 2** → Section 4.3, Baseline Comparison
- **Phase 3** → Section 3.5, Algorithm 1 & 2
- **Phase 4** → Section 4.5, Figure 3; Section 4.6, Case Study

---

## 🎓 引用

如使用此實作，請引用原論文：
```
A Knowledge Graph-Enhanced Large Language Model Framework 
for Goat Disease Question Answering System
```

---

# ✅ 執行前檢查清單 (Pre-execution Checklist)

在開始實驗前，請確認以下項目：

## 1. 環境檢查
```bash
# 檢查 Neo4j 是否啟動
# 瀏覽器訪問: http://localhost:7474

# 檢查 Ollama 是否啟動
ollama list

# 確認模型已下載
ollama pull deepseek-r1:8b-llama-distill-q4_K_M
ollama pull nomic-embed-text:nomic
```

## 2. 檔案檢查
- ✅ `goat_data_text collection-1.2-eng.txt` 存在
- ✅ `topic.csv` 存在

## 3. 依賴套件檢查
```python
# 執行此 Cell 檢查套件是否安裝
import sys
required_packages = [
    'neo4j', 'pandas', 'numpy', 'scikit-learn', 
    'matplotlib', 'seaborn', 'neo4j_graphrag'
]

missing = []
for pkg in required_packages:
    try:
        __import__(pkg.replace('-', '_'))
        print(f"✅ {pkg}")
    except ImportError:
        print(f"❌ {pkg} - 請執行: pip install {pkg}")
        missing.append(pkg)

if missing:
    print(f"\n⚠️ 請先安裝缺少的套件: pip install {' '.join(missing)}")
else:
    print("\n✅ 所有依賴套件已安裝！")
```

## 4. 執行順序提醒
1. 從上到下依序執行所有 Cell
2. Phase 1 會花費較長時間（約 30-60 分鐘）
3. Phase 4 會產生大量日誌檔案
4. 建議在執行前先備份現有的 Neo4j 資料庫

---

**準備好了嗎？從第二個 Cell 開始執行吧！** 🚀